<img src="resources/cropped-SummerWorkshop_Header.png">  

<h1 align="center">Workshop SWDB 2024 </h1> 
<h3 align="center">Day 3 2024 - Neuron Morphology</h3> 
<h3 align="center">Notebook 4: Analyzing Brain Connectivity via Projections of Light Microscopy Neurons</h3> 

This is the color key that i'm using for leaving comments...

<font color='green'><b> Green<b></font> --> Add image
    
<font color='orange'><b> Orange<b></font> --> Write up the thing being described
    
<font color='red'><b> Red<b></font> --> Question

<font color='purple'><b> Purple<b></font> --> Feedback
    

<b> Note: This beginning part is very wordy and most of the text is probably unnecssary. Let's trim it down in the next version, it's here now so that you have an idea of what type of analysis this notebook is geared towards. 

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
    
# Section 1: Introduction


<font size="3.5"> The main objective of this notebook is to analyze the "projections" of Light Microscopy (LM) neurons, meaning the regions of the brain that the axons and dendrites traverse and communicate their inputs/outputs via the endpoints. This type of long-range projection analysis is only applicable for the LM neurons since the EM neurons were reconstructed within a small piece of tissue fully contained within the visual cortex. In contrast, we will see that many LM neurons have axons that project across several brain regions, see Figure X.  </font>


<font size="3" color='green'><b> Insert image of exaspim brain that shows maybe a few neurons that clearly project a long distance to emphasize the complexity of neural projections. </font>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<font size="3.5"> As an introduction to analyzing the projections of LM neurons, we will explore two open-ended questions related to brain connectivity that scientists consider when analyzing LM neurons. These questions aim to uncover both the fundamental connections and complex network dynamics that define the functional roles of these neurons within the broader neural circuitry. </font>

<font size="3.5"> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <strong>Question 1:</strong> What do the inputs to a particular brain region look like? Where else do those inputs send their collaterals? </font>

<font size="3.5"> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <strong>Question 2:</strong> Are neuronal morphologies stereotyped, or are there distinct projection classes?  </font>

<font size="3.5"> Both of these questions involve analyzing a large number of neurons to provide a general overview of how neurons <em>connect</em> different brain regions.  Before addressing these questions, we must first understand the meaning of <em>connectivity</em> at the level of a single neuron, then generalize this notion to the level of brain regions. In this context a neuron connects regions <em>A</em> and <em>B</em> if the neuron has dendritic endpoints in region <em>A</em> and axonal endpoints in region <em>B</em>. The dendritic endpoints receive the <em>input</em> of a neuron, whereas the axonal endpoints send the neuron's output.</font>

<font size="3.5" color='orange'><b> Explain inputs and outputs of a brain region </font>


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<font size="5"> Section 1.1: Strategy for Answering Scientific Questions </font>

<font size="3.5"> The first objective of this notebook is to walk through how to answer Question 1 by analyzing neurons from the exapsim dataset. Once we are able to answer the first question, we will have developed the necessary tools to effectively analyze and explore the complexities involved in Question 2. </font>

<font size="3.5" color='orange'><b> Explain what question 1 is asking in a simple and intuitive way. </font></b>

<font size="3.5"> In order to answer the first question, we need to be able to extract the following information from neurons in the exaspim dataset...  </font>


<font size="3.5"><strong> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Task 1:</strong> Which brain regions do the dendritic or axonal endpoints of a given neuron belong to? </font>

<font size="3.5"><strong>  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Task 2:</strong> Find all neurons that have dendritic or axonal endpoints in a given brain region. </font>


<font size="3.5" color='orange'><b> Explain how to analyze this information to answer question 1. </font><b>

<font size="3.5" color='orange'><b> Section 2 explains how to do 1 and Section 3 explains how to do 2. </font><b>



<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<font size="5"> Section 1.2: Analyzing Connectivity with the Common Coordinate Framework (CCF) </font> 

<font size="3.5"> In order to analyze the projections of these neurons, each brain needs to be registered to a standardized template brain space referred to as the Common Coordinate Framework (CCF), see Figure Y. This registration step is important because it enables scientists to analyze and compare neuron reconstructions from multiple brains in an integrated framework. </font>

<font size="3.5" color='red'><b> What should we say about the ccf? </font><b>


<font size="3" color='green'><b> Insert image of CCF </font><b>

In [1]:
import pandas as pd


def get_ccf_name(ccf_id):
    idx = ccf_structures["id"] == ccf_id
    try:
        return ccf_structures.loc[idx, "name"].iloc[0]
    except:
        return ccf_id


# Load the CCF structure data as a DataFrame
ccf_structures = pd.read_csv('/data/adult_mouse_ccf_structures.csv')
ccf_structures.head()

,id,name,acronym,hemisphere_id,parent_structure_id,graph_order,structure_id_path,color_hex_triplet
0,1000,extrapyramidal fiber systems,eps,3,1009.0,1218,/997/1009/1000/,CCCCCC
1,223,Arcuate hypothalamic nucleus,ARH,3,157.0,733,/997/8/343/1129/1097/157/223/,FF5D50
2,12998,"Somatosensory areas, layer 6b",SS6b,3,453.0,36,/997/8/567/688/695/315/453/12998/,188064
3,163,"Agranular insular area, posterior part, layer 2/3",AIp2/3,3,111.0,287,/997/8/567/688/695/315/95/111/163/,219866
4,552,"Pontine reticular nucleus, ventral part",PRNv,3,987.0,914,/997/8/343/1065/771/987/552/,FFBA86


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<font size="3.5"> The ccf ids are stored as integers in a meshparty skeleton, so we'll use the function "get_ccf_name" to get the name of the region corresponding to a given id. Here is a simple example of using this function: </font>

In [2]:
ccf_id = 12998
print(f"The CCF ID '{ccf_id}' represents the '{get_ccf_name(ccf_id)}'\n")


The CCF ID '12998' represents the 'Somatosensory areas, layer 6b'



<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
    
# Section 2: Analyzing Connectivity of a Single Neuron
    
<font size="3.5"> Let's load the skeleton dataset of LM neurons, then sample a single skeleton and analyze what regions of the brain this neuron connects. </font>

In [3]:
# Imports
from random import sample
from utils import graph_utils

import cloudvolume
import numpy as np


# Initializations
input_directory = "precomputed://s3://aind-open-data/exaSPIM_609281_2022-11-03_13-49-18_reconstructions/precomputed"
skel_cv_dataset = cloudvolume.CloudVolume(input_directory)
skel_ids = [1, 2, 3, 4, 5, 7, 9, 14] + np.arange(16, 33).tolist()

print("Overview of LM Neuron Dataset...")
print("# Skeletons:", len(skel_ids))
print("Skeleton IDs:", skel_ids)

# Sample single skeleton
skel_id = sample(skel_ids, 1)[0]
skel = graph_utils.get_skeleton(skel_cv_dataset, skel_id)
print(f"\nLoaded Skeleton-{skel_id}\n")


Overview of LM Neuron Dataset...
# Skeletons: 25
Skeleton IDs: [1, 2, 3, 4, 5, 7, 9, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]

Loaded Skeleton-16



<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<font size="3.5"> Each skeleton has a node-level attribute called "id" that specifies where a node is located in ccf space. We will use subroutine called "get_ccf_ids" which is stored in "graph_utils.py". The purpose of this routine is to easily extract the ccf ids from vertices within a certain compartment (e.g. axons or dendrites) and/or vertices which are end points or branch points, see documentation for more details. Next, let's look at some simple examples of using the routine "get_ccf_ids". </font>

In [4]:
# Root - CCF Compartment
soma_ccf = graph_utils.get_ccf_ids(skel, compartment_type=1)
print("Soma is in the", get_ccf_name(soma_ccf[0]))

# Axon - CCF Regions
print("\nAxons Traverse these CCF Regions...")
axons_ccf = graph_utils.get_ccf_ids(skel, compartment_type=3)
for ccf_id in np.unique(axons_ccf):
    print("  ", get_ccf_name(ccf_id))

# Dendrite - CCF Compartments
print("\nDendrites Traverse these CCF Regions...")
dendrites_ccf = graph_utils.get_ccf_ids(skel, compartment_type=2)
for ccf_id in np.unique(dendrites_ccf):
    print("  ", get_ccf_name(ccf_id))
    

Soma is in the Gigantocellular reticular nucleus

Axons Traverse these CCF Regions...
   Intermediate reticular nucleus
   Nucleus raphe obscurus
   Magnocellular reticular nucleus
   Parvicellular reticular nucleus
   Paragigantocellular reticular nucleus, dorsal part
   Paragigantocellular reticular nucleus, lateral part
   crossed tectospinal pathway
   Gigantocellular reticular nucleus

Dendrites Traverse these CCF Regions...
   Medulla
   Gigantocellular reticular nucleus
   Medullary reticular nucleus, ventral part
   nan


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">


<font size="3.5"> Lets compute the ccf region distriubtion for the axonal and dendritic endpoints. These statistics are more meaningful because they describe where neurons receive/send inputs/outputs. We can use that same routine "get_ccf_ids" to more easily get these ccf ids. </font>


In [5]:
# Subroutines
def report_distribution(values, percent_threshold=0):
    ids, cnts = np.unique(values, return_counts=True)
    print("% Vertices   CCF Region")
    for idx in np.argsort(-cnts):
        percent = 100 * cnts[idx] / len(values)
        ccf_id = get_ccf_name(ids[idx])
        if percent >= percent_threshold:
            print(f"{round(percent, 2)}      {ccf_id}")


# Axon Endpoints - CCF Regions
print("\nDistribution of CCF Regions of Axon Endpoints...")
axon_endpoints_ccf = graph_utils.get_ccf_ids(skel, compartment_type=3, vertex_type="end_points")
report_distribution(axon_endpoints_ccf)

# Dendrite Endpoints - CCF Regions
print("\nDistribution of CCF Regions of Dendrite Endpoints...")
dendrite_endpoints_ccf = graph_utils.get_ccf_ids(skel, compartment_type=2, vertex_type="end_points")
report_distribution(dendrite_endpoints_ccf)



Distribution of CCF Regions of Axon Endpoints...
% Vertices   CCF Region
38.46      Intermediate reticular nucleus
38.46      Gigantocellular reticular nucleus
7.69      Magnocellular reticular nucleus
5.13      Parvicellular reticular nucleus
5.13      Paragigantocellular reticular nucleus, lateral part
2.56      Nucleus raphe obscurus
2.56      Paragigantocellular reticular nucleus, dorsal part

Distribution of CCF Regions of Dendrite Endpoints...
% Vertices   CCF Region
98.03      nan
1.32      Medullary reticular nucleus, ventral part
0.66      Gigantocellular reticular nucleus


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
    
# Section 2: Analyzing Connectivity Between Brain Regions

<font size='3.5'> Next, let's load all of the skeletons in the dataset.</font>

In [7]:
# Load all skeletons in dataset
skels = dict((skel_id, graph_utils.get_skeleton(skel_cv_dataset, skel_id)) for skel_id in skel_ids)


In [9]:
# Sample ccf_id and extract skeletons with soma with that ccf region
ccf_id = sample(np.unique(soma_ccf).tolist(), 1)[0]
skels_subset = [skel for skel in skels.values() if skel.vertex_properties['ccf'][skel.root] == ccf_id]
print(f"# Skeletons with Soma in {get_ccf_name(ccf_id)}:", len(skels_subset))

# Get ccf regions of endpoints
axon_endpoints_ccf = list()
dendrite_endpoints_ccf = list()
for skel in skels_subset:
    axon_endpoints_ccf.extend(graph_utils.get_ccf_ids(skel, compartment_type=3, vertex_type="end_points").tolist())
    dendrite_endpoints_ccf.extend(graph_utils.get_ccf_ids(skel, compartment_type=2, vertex_type="end_points").tolist())

# Report distribution
print("\nDistribution of Axon Endpoints in CCF Space...")
report_distribution(axon_endpoints_ccf)

print("\nDistribution of Dendrite Endpoints in CCF Space...")
report_distribution(dendrite_endpoints_ccf, percent_threshold=0.5)


# Skeletons with Soma in Gigantocellular reticular nucleus: 9

Distribution of Axon Endpoints in CCF Space...
% Vertices   CCF Region
56.2      Gigantocellular reticular nucleus
17.87      Intermediate reticular nucleus
6.05      crossed tectospinal pathway
4.61      Paragigantocellular reticular nucleus, dorsal part
4.03      Medulla
3.46      Magnocellular reticular nucleus
1.15      Medullary reticular nucleus, ventral part
0.86      Nucleus raphe obscurus
0.86      Paragigantocellular reticular nucleus, lateral part
0.86      Lateral reticular nucleus, magnocellular part
0.86      Nucleus raphe magnus
0.86      medial longitudinal fascicle
0.58      Nucleus prepositus
0.58      Parvicellular reticular nucleus
0.58      Linear nucleus of the medulla
0.29      Pons
0.29      facial nerve

Distribution of Dendrite Endpoints in CCF Space...
% Vertices   CCF Region
15.18      Periaqueductal gray
15.1      nan
8.55      Midbrain
8.53      Lateral mammillary nucleus
5.54      Midbrain ret